In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
from pathlib import Path
import gc
from zipfile import ZipFile
from unidecode import unidecode

project_dir = Path("notebook.ipynb").resolve().parents[0]

In [2]:
# lẽ todos os arquivos .zip e descompacta na pasta raw_data:
for archive in os.listdir(f'{project_dir}/raw_data'):
    with ZipFile(f'{project_dir}/raw_data/{archive}','r') as zipfile:
        zipfile.extractall(f'{project_dir}/data/')

total=0

gc.collect()

#transforma os dados da forma wide para forma long, além de padronizar nomes de colunas:
for file in os.listdir(f'{project_dir}/data/'):
    if file == 'TarifaMediaFornecimento.csv':
        continue
    else:
        data = pd.read_csv(f'{project_dir}/data/{file}',sep='\n', encoding='latin-1', header=None)
        data = data[0].str.split(';', expand=True)
        data = data.reset_index(drop=True)
        var_name = data[0][1][7:]
        var_name = unidecode(var_name).lower().strip().replace(' ','_').replace('/','_').replace(':','')
        data.columns = data.iloc[5].tolist()
        data = data.drop([0,1,2,3,4,5], axis=0).reset_index(drop=True)
        data = data.drop(columns=['latitude','longitude'])
        for column in data.columns[2:]:
            column_name = re.sub(r'\D*','',column)
            data = data.rename(columns={column:f'{column_name}'})
        
        data_melted = data.melt(id_vars=['Município','ibge'], var_name=f'ano', value_name=f'{var_name}_value')
        data_melted.to_csv(f'{project_dir}/melted_data/{var_name}_value.csv', sep=';', decimal=',', encoding='latin-1', index=False)
    total+= 1

melted_data = pd.read_csv(f'{project_dir}/melted_data/__educacao_ensino_fundamental_taxa_de_abandono_total_value.csv', header=None, sep='\n',encoding='latin-1')
melted_data = melted_data[0].str.split(';', expand=True)
melted_data.columns= melted_data.iloc[0]
melted_data = melted_data.drop([0], axis=0)
melted_data = melted_data.astype({'ibge':'str','ano':'str'})

gc.collect()


#concatena todos os arquivos em um único dataset:
for file in os.listdir(f'{project_dir}/melted_data/'):
    if file =='__educacao_ensino_fundamental_taxa_de_abandono_total_value.csv':
        continue
    df= pd.read_csv(f'{project_dir}/melted_data/{file}', header=None, sep='\n', encoding='latin-1')
    df = df[0].str.split(';', expand=True)
    df.columns = df.iloc[0]
    df = df.drop([0], axis=0)
    df = df.astype({'ibge':'str','ano':'str'})
    melted_data = melted_data.merge(df, on=['Município','ibge','ano'], suffixes=('','_D'), how='outer')
    melted_data = melted_data.drop(melted_data.filter(regex='_D$').columns.tolist(), axis=1)
    melted_data.to_csv(f'{project_dir}/preliminar_dataset.csv', sep=';',decimal=',', encoding='latin-1')

final_dataset = melted_data
final_dataset.to_csv(f'{project_dir}/final_dataset.csv', sep=';',decimal=',', encoding='latin-1')
print(f'Tamanho final: {final_dataset.shape}')

Tamanho final: (22497, 173)


In [5]:
final_dataset.shape

(22497, 173)